# Setup

In [1]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OPEN API KEY: ")
os.environ["TAVILY_API_KEY"] = getpass("TAVILY API KEY: ")

OPEN API KEY:  ········
TAVILY API KEY:  ········


Set up Langsmith tracing credentials

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass("LangSmith API Key:")

LangSmith API Key: ········


# Set up tools

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolExecutor

tools = [TavilySearchResults(max_results=1)]
tool_executor = ToolExecutor(tools)

# Setup model

In [5]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI(temperature=0, streaming=True)

Convert langchain tools to openai function calling and binding them to the model

In [7]:
from  langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model_with_tools = model.bind_functions(functions)

# Define the agent state

In [9]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages.base import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# Define the nodes

In [21]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that calls the model
def run_agent(state):
    messages = state["messages"]
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

# Define the function that calls the tool
def invoke_tool(state):
    messages = state["messages"]
    # We know last message involves function call
    last_message = messages[-1]
    # We construct ToolInvocation from function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"])
    )
    # Call the tool executor
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    return {"messages": [function_message]}
    
# Define function for handling conditional edge
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    if "function_call" in last_message.additional_kwargs:
        return "continue"
        
    return "end"

# Define the graph

In [22]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

# Define two nodes, agent and action
workflow.add_node("agent", run_agent)
workflow.add_node("action", invoke_tool)

# Set the entry point
workflow.set_entry_point("agent")

# Create a conditional edge for "agent"
# If there is function_call, state flows from agent to action node, else flow ends
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

# Define normal edge
workflow.add_edge("action", "agent")

# Compile the workflow
app = workflow.compile()

# Use tool

In [23]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="What is the weather in Lagos, Nigeria")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='What is the weather in Lagos, Nigeria'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Lagos, Nigeria"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}),
  FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in Lagos, Nigeria is {\'location\': {\'name\': \'Lagos\', \'region\': \'Lagos\', \'country\': \'Nigeria\', \'lat\': 6.45, \'lon\': 3.4, \'tz_id\': \'Africa/Lagos\', \'localtime_epoch\': 1712181708, \'localtime\': \'2024-04-03 23:01\'}, \'current\': {\'last_updated_epoch\': 1712181600, \'last_updated\': \'2024-04-03 23:00\', \'temp_c\': 28.0, \'temp_f\': 82.4, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 170, \'wind_dir\': \'S\', \'pressure_mb\': 1010.0, \'pressure_